# Distributed HPO with Ray Tune and XGBoost-Ray

This demo introduces **Ray tune's** key concepts using a classification example. This example is derived from [Hyperparameter Tuning with Ray Tune and XGBoost-Ray](https://github.com/ray-project/xgboost_ray#hyperparameter-tuning). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

 1. Setup your config space and define your trainable and objective function
 2. Use Tune to execute your training hyperparameter sweep, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#summary) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine or analyse the results returned
 
 <img src="https://docs.ray.io/en/latest/_images/tune-workflow.png" height="50%" width="60%">


See also the [Understanding Hyperparameter Tuning](https://github.com/anyscale/academy/blob/main/ray-tune/02-Understanding-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


In [1]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

import ray
from ray import tune
CONNECT_TO_ANYSCALE=True

In [2]:
if ray.is_initialized:
    ray.shutdown()
    if CONNECT_TO_ANYSCALE:
        ray.init("anyscale://jsd-weekly-demo")
    else:
        ray.init()

Authenticating
Loaded Anyscale authentication token from ANYSCALE_CLI_TOKEN.

Output
(anyscale +0.6s) WARNING: No working_dir specified! Files will only be uploaded to the cluster if a working_dir is provided or a project is detected. In the future, files will only be uploaded if working_dir is provided. To ensure files continue being imported going forward, set the working_dir in your runtime environment. See https://docs.ray.io/en/latest/handling-dependencies.html#runtime-environments.
(anyscale +0.3s) .anyscale.yaml found in project_dir. Directory is attached to a project.
(anyscale +0.4s) Using project (name: prj-weekly-demo, project_dir: /Users/jules/git-repos/ray-core-tutorial, id: prj_5rvR1w2ciyUs9RM27FeZ6FVB).
(anyscale +1.6s) cluster jsd-weekly-demo is currently running, the cluster will not be restarted.


2022-01-20 17:52:54,973	INFO packaging.py:352 -- Creating a file package for local directory '/Users/jules/git-repos/ray-core-tutorial'.
2022-01-20 17:52:55,013	INFO packaging.py:221 -- Pushing file package 'gcs://_ray_pkg_660df966c5072f1d.zip' (6.24MiB) to Ray cluster...
2022-01-20 17:52:55,770	INFO packaging.py:224 -- Successfully pushed file package 'gcs://_ray_pkg_660df966c5072f1d.zip'.


(anyscale +12.4s) Connected to jsd-weekly-demo, see: https://console.anyscale.com/projects/prj_5rvR1w2ciyUs9RM27FeZ6FVB/clusters/ses_jUg93ra8KHWTzAMZv5nig2Rb
(anyscale +12.4s) URL for head node of cluster: https://session-jug93ra8khwtzamzv5nig2rb.i.anyscaleuserdata.com


## Step 1: Define a 'Trainable' training function to use with Ray Tune `ray.tune(...)`

In [3]:
NUM_OF_ACTORS = 4           # degree of parallel trials; each actor will have a separate trial with a set of unique config from the search space
NUM_OF_CPUS_PER_ACTOR = 1   # number of CPUs per actor

ray_params = RayParams(num_actors=NUM_OF_ACTORS, cpus_per_actor=NUM_OF_CPUS_PER_ACTOR)

In [4]:
def train_func_model(config:dict, checkpoint_dir=None):
    # create the dataset
    train_X, train_y = load_breast_cancer(return_X_y=True)
    # Convert to RayDMatrix data structure
    train_set = RayDMatrix(train_X, train_y)

    # Empty dictionary for the evaluation results reported back
    # to tune
    evals_result = {}

    # Train the model with XGBoost train
    bst = train(
        params=config,                       # our hyperparameter search space
        dtrain=train_set,                    # our RayDMatrix data structure
        evals_result=evals_result,           # place holder for results
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)                # distributed parameters configs for Ray Tune

    bst.save_model("model.xgb")

## Step 2: Define a hyperparameter search space

In [5]:
 # Specify the typical hyperparameter search space
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

## Step 3: Run Ray tune main trainer and examine the results

Ray Tune will launch distributed HPO, using four remote actors, each with its own instance of the trainable func

<img src="images/ray_tune_dist_hpo.png" height="60%" width="70%"> 

In [6]:
# Run tune
analysis = tune.run(
    train_func_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    verbose=1,
    resources_per_trial=ray_params.get_tune_resources()
)

(run pid=2000) == Status ==
(run pid=2000) Current time: 2022-01-20 17:53:02 (running for 00:00:00.12)
(run pid=2000) Memory usage on this node: 2.6/62.0 GiB
(run pid=2000) Using FIFO scheduling algorithm.
(run pid=2000) Resources requested: 0/80 CPUs, 0/0 GPUs, 0.0/216.23 GiB heap, 0.0/91.79 GiB objects
(run pid=2000) Result logdir: /home/ray/ray_results/train_func_model_2022-01-20_17-53-02
(run pid=2000) Number of trials: 4/4 (4 PENDING)
(run pid=2000) 
(run pid=2000) 


(ImplicitFunc pid=2058) 2022-01-20 17:53:04,556	INFO main.py:979 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=444, ip=172.31.10.165) 2022-01-20 17:53:04,591	INFO main.py:979 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=438, ip=172.31.6.246) 2022-01-20 17:53:04,605	INFO main.py:979 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=407, ip=172.31.30.171) 2022-01-20 17:53:04,751	INFO main.py:979 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=2058) 2022-01-20 17:53:06,374	INFO main.py:1024 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=2121) [17:53:06] task [xgboost.ray]:139651882759504 got new rank 3
(_RemoteRayXGBoostActor pid=2120) [17:53:06] task [xgboost.ray]:140483010139520 got ne

(run pid=2000) 2022-01-20 17:53:07,121	WARN commands.py:269 -- Loaded cached provider configuration
(run pid=2000) 2022-01-20 17:53:07,122	WARN commands.py:270 -- If you experience issues with the cloud provider, try re-running the command with --no-config-cache.
(run pid=2000) Authenticating
(run pid=2000) Loaded Anyscale authentication token from variable.
(run pid=2000) 
(run pid=2000) 2022-01-20 17:53:08,063	INFO command_runner.py:357 -- Fetched IP: 172.31.10.165
(run pid=2000) 2022-01-20 17:53:08,063	INFO log_timer.py:25 -- NodeUpdater: ins_YBu42WCHAu7YjxBHFasB4NL4: Got IP  [LogTimer=43ms]
(run pid=2000) == Status ==
(run pid=2000) Current time: 2022-01-20 17:53:08 (running for 00:00:06.44)
(run pid=2000) Memory usage on this node: 3.2/62.0 GiB
(run pid=2000) Using FIFO scheduling algorithm.
(run pid=2000) Resources requested: 20.0/80 CPUs, 0/0 GPUs, 0.0/216.23 GiB heap, 0.0/91.79 GiB objects
(run pid=2000) Current best trial: ddfe6_00002 with train-error=0.080844 and parameters={

(ImplicitFunc pid=444, ip=172.31.10.165) 2022-01-20 17:53:11,973	INFO main.py:1503 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 7.42 seconds (5.56 pure XGBoost training time).
(ImplicitFunc pid=2058) 2022-01-20 17:53:11,978	INFO main.py:1503 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 7.50 seconds (5.60 pure XGBoost training time).
(ImplicitFunc pid=407, ip=172.31.30.171) 2022-01-20 17:53:11,986	INFO main.py:1503 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 7.28 seconds (5.21 pure XGBoost training time).
(ImplicitFunc pid=438, ip=172.31.6.246) 2022-01-20 17:53:11,991	INFO main.py:1503 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 7.43 seconds (5.56 pure XGBoost training time).
(run pid=2000) 2022-01-20 17:53:12,147	INFO tune.py:626 -- Total run time: 9.99 seconds (9.65 seconds for the tuning loop).


(run pid=2000) == Status ==
(run pid=2000) Current time: 2022-01-20 17:53:12 (running for 00:00:09.66)
(run pid=2000) Memory usage on this node: 3.0/62.0 GiB
(run pid=2000) Using FIFO scheduling algorithm.
(run pid=2000) Resources requested: 0/80 CPUs, 0/0 GPUs, 0.0/216.23 GiB heap, 0.0/91.79 GiB objects
(run pid=2000) Current best trial: ddfe6_00001 with train-error=0.012302 and parameters={'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.012907321821872259, 'subsample': 0.9931528239659944, 'max_depth': 7, 'nthread': 1, 'n_jobs': 1}
(run pid=2000) Result logdir: /home/ray/ray_results/train_func_model_2022-01-20_17-53-02
(run pid=2000) Number of trials: 4/4 (4 TERMINATED)
(run pid=2000) 
(run pid=2000) 


In [7]:
print("Best hyperparameters", analysis.best_config)

Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.012907321821872259, 'subsample': 0.9931528239659944, 'max_depth': 7}


In [8]:
analysis.results_df.head(5)

/usr/local/anaconda3/envs/ray-core/lib/python3.8/site-packages/ray/tune/analysis/experiment_analysis.py:262: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,train-logloss,train-error,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,...,iterations_since_restore,experiment_tag,config.tree_method,config.objective,config.eval_metric,config.eta,config.subsample,config.max_depth,config.nthread,config.n_jobs
trial_id,,,,,,,,,,,,,,,,,,,,,
ddfe6_00000,0.673668,0.014060,0.007867,True,None,None,10,b52bf0dfc1d44928862bb72a2fa73492,2022-01-20_17-53-11,1642729991,...,10,"0_eta=0.0022943,max_depth=8,subsample=0.73505",approx,binary:logistic,"[logloss, error]",0.002294,0.735052,8,1,1
ddfe6_00001,0.589173,0.012302,0.005719,True,None,None,10,0c4561955cff482699ef8ab9ea4f53a2,2022-01-20_17-53-11,1642729991,...,10,"1_eta=0.012907,max_depth=7,subsample=0.99315",approx,binary:logistic,"[logloss, error]",0.012907,0.993153,7,1,1
ddfe6_00002,0.671114,0.075571,0.007623,True,None,None,10,6f237e5fa2b047ceb59ab1b7284edc79,2022-01-20_17-53-11,1642729991,...,10,"2_eta=0.0032207,max_depth=1,subsample=0.78223",approx,binary:logistic,"[logloss, error]",0.003221,0.782226,1,1,1
ddfe6_00003,0.681504,0.015817,0.007399,True,None,None,10,d58cd5b10f8f43d1ab59a2129d86ccc4,2022-01-20_17-53-11,1642729991,...,10,"3_eta=0.0013646,max_depth=8,subsample=0.71926",approx,binary:logistic,"[logloss, error]",0.001365,0.719262,8,1,1


---

In [9]:
ray.shutdown()

## References

 * [Ray Train: Tune: Scalable Hyperparameter Tuning](https://docs.ray.io/en/master/tune/index.html)
 * [Introducing Distributed XGBoost Training with Ray](https://www.anyscale.com/blog/distributed-xgboost-training-with-ray)
 * [How to Speed Up XGBoost Model Training](https://www.anyscale.com/blog/how-to-speed-up-xgboost-model-training)
 * [XGBoost-Ray Project](https://github.com/ray-project/xgboost_ray)
 * [Distributed XGBoost on Ray](https://docs.ray.io/en/latest/xgboost-ray.html)